<a href="https://colab.research.google.com/github/NelaRepko/NelaRepko-GoIT_hw_Python/blob/main/goit_hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving industries(2.0).csv to industries(2.0).csv
Saving applications(2.0).csv to applications(2.0).csv


In [2]:
uploaded.keys()

dict_keys(['industries(2.0).csv', 'applications(2.0).csv'])

In [3]:
import pandas as pd

df_applications = pd.read_csv('applications(2.0).csv')
df_industries = pd.read_csv('industries(2.0).csv')

In [4]:
# Прибираємо дублікати по стовпцю 'applicant_id', залишаємо перший запис
df_applications = df_applications.drop_duplicates(subset='applicant_id', keep='first')
print(f"Кількість рядків після видалення дублікатів: {len(df_applications)}")

Кількість рядків після видалення дублікатів: 13278


In [5]:
# Заповнюємо пропущені значення у стовпці 'External Rating' нулями
df_applications['External Rating'] = df_applications['External Rating'].fillna(0)
print(df_applications['External Rating'].isna().sum())

0


In [6]:
# Заповнюємо пропущені значення у стовпці 'Education level' текстом "Середня"
df_applications['Education level'] = df_applications['Education level'].fillna("Середня")
print(df_applications['Education level'].isna().sum())

0


In [7]:
# Об'єднуємо df_applications та df_industries по стовпцю 'Industry'
df_applications = df_applications.merge(df_industries, on='Industry', how='left')
# Перевірка:
df_applications.head()

,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id,Score
0,11.30.2022 10:26:37,12000.0,29,Чоловік,Blockchain,Other,8.0,"Вища (бакалавр, спеціаліст, магістр)",Івано-Франківськ чи область,99e7b0dc6cc05dd334d8f38dc26ce9b3,0
1,11.30.2022 10:26:39,NaN,36,Чоловік,Public services / Government,Single,3.0,"Вища (бакалавр, спеціаліст, магістр)",NaN,63dfcf8e6904186650d6814279fbe42f,20
2,11.30.2022 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498,10
3,11.30.2022 10:27:31,1500.0,23,Жінка,Telecom,Single,0.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,5847ac62cc9eac5e323c2517dcc91ad1,15
4,11.30.2022 10:27:34,8400.0,33,Жінка,Automotive,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Житомир чи область,5d21f3795b50de8e8f8f8d5f48b754f3,15


In [17]:
df_applications.info()

# перші 10 значень стовпця Score
print("\nPreview Score (first 10):")
print(df_applications['Score'].head(10))

# Тип даних стовпця перед перетворенням
print("\nCurrent dtype of Score:", df_applications['Score'].dtype)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13278 entries, 0 to 13277
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Applied at       13278 non-null  object 
 1   Amount           13245 non-null  float64
 2   Age              13278 non-null  int64  
 3   Gender           13278 non-null  object 
 4   Industry         13278 non-null  object 
 5   Marital status   13278 non-null  object 
 6   External Rating  13278 non-null  float64
 7   Education level  13278 non-null  object 
 8   Location         11506 non-null  object 
 9   applicant_id     13278 non-null  object 
 10  Score            13278 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 1.1+ MB

Preview Score (first 10):
0     0
1    20
2    10
3    15
4    15
5    15
6     5
7    15
8    15
9    10
Name: Score, dtype: int64

Current dtype of Score: int64


In [18]:
import pandas as pd
from datetime import datetime

# Створюємо копію датафрейму, щоб не псувати оригінал
df = df_applications.copy()

# Переконаймося, що поле 'Applied at' у форматі дати
df['Applied at'] = pd.to_datetime(df['Applied at'], errors='coerce')

# Початкове значення рейтингу = 0
df['Rating'] = 0

# Умова 1: Вік від 35 до 55 → +20
df.loc[(df['Age'] >= 35) & (df['Age'] <= 55), 'Rating'] += 20

# Умова 2: Заявка подана не у вихідні → +20
df.loc[df['Applied at'].dt.dayofweek < 5, 'Rating'] += 20   # 0-4 = будні

# Умова 3: Одружений → +20
df.loc[df['Marital status'].str.lower() == 'married', 'Rating'] += 20

# Умова 4: Київ або область → +10
df.loc[df['Location'].str.contains('київ', case=False, na=False), 'Rating'] += 10

#Умова 5: Додаємо Score з industries.csv
df['Rating'] += df['Score']

# Умова 6: External Rating ≥ 7 → +20
df.loc[df['External Rating'] >= 7, 'Rating'] += 20

# External Rating ≤ 2 → -20
df.loc[df['External Rating'] <= 2, 'Rating'] -= 20

# Якщо 'Amount' відсутній або 'External Rating' == 0 → Rating = 0
df.loc[(df['Amount'].isna()) | (df['External Rating'] == 0), 'Rating'] = 0

# Обмежимо рейтинг від 0 до 100
df['Rating'] = df['Rating'].clip(lower=0, upper=100)

# Перевірка:
df[['applicant_id', 'Age', 'Applied at', 'Marital status', 'Location', 'Industry', 'External Rating', 'Score', 'Rating']].head(5)

,applicant_id,Age,Applied at,Marital status,Location,Industry,External Rating,Score,Rating
0,99e7b0dc6cc05dd334d8f38dc26ce9b3,29,2022-11-30 10:26:37,Other,Івано-Франківськ чи область,Blockchain,8.0,0,40
1,63dfcf8e6904186650d6814279fbe42f,36,2022-11-30 10:26:39,Single,NaN,Public services / Government,3.0,20,0
2,10dbafaeb46c09e96b6987c03bbb3498,34,2022-11-30 10:26:58,Single,Київ чи область,Adtech / Advertising,4.0,10,40
3,5847ac62cc9eac5e323c2517dcc91ad1,23,2022-11-30 10:27:31,Single,Львів чи область,Telecom,0.0,15,0
4,5d21f3795b50de8e8f8f8d5f48b754f3,33,2022-11-30 10:27:34,Single,Житомир чи область,Automotive,6.0,15,35


In [19]:
# Залишаємо лише заявки з рейтингом більше нуля
df_accepted = df[df['Rating'] > 0].copy()
# Перевірка:
print(f"Кількість прийнятих заявок: {len(df_accepted)}")

Кількість прийнятих заявок: 12088


In [24]:
# Перетворюємо 'Applied at' на datetime
df_accepted['Applied at'] = pd.to_datetime(df_accepted['Applied at'], format='%m.%d.%Y %H:%M:%S')
# Додаємо колонку з номером тижня
df_accepted['Week'] = df_accepted['Applied at'].dt.isocalendar().week
# Групуємо по тижню та рахуємо середній рейтинг
weekly_avg_rating = df_accepted.groupby('Week')['Rating'].mean().reset_index()
# округлюєм
weekly_avg_rating['Rating'] = weekly_avg_rating['Rating'].round(2)
# Виводимо результат
print(weekly_avg_rating)

   Week  Rating
0    48   53.47
1    50   48.14
2    51   46.96
3    52   51.04
